In [2]:
import rasterio  # Mengimpor modul rasterio untuk membaca dan memproses data raster
import numpy as np  # Mengimpor modul numpy untuk operasi numerik
import pandas as pd  # Mengimpor modul pandas untuk analisis data tabular
from rasterio import plot  # Mengimpor fungsi plot dari modul rasterio
from rasterio.transform import Affine  # Mengimpor kelas Affine dari modul rasterio.transform

# Definisikan sistem referensi koordinat (CRS) yang digunakan dalam data raster
# Definisikan transformasi affine untuk data raster
crs = 'EPSG:4326'  
transform = Affine(0.01, 0, 0, 0, -0.01, 0)  

# Tentukan lokasi file GeoTIFF pertama
filepath_citra1 = 'TIF koreksi/2000_CART_modif2.tif'  

# Buka file GeoTIFF pertama dengan mode baca menggunakan rasterio.open() dan atur CRS serta transformasinya
# Baca band pertama dari file raster sebagai float
with rasterio.open(filepath_citra1, 'r', driver='GTiff', crs=crs, transform=transform) as src1:
    raster_t1 = src1.read(1).astype(float)  

# Tentukan nama file untuk menyimpan data raster pertama sebagai teks
inputfile_t1 = 't1.txt'  

# Simpan matriks NumPy sebagai file teks
np.savetxt(inputfile_t1, raster_t1, fmt='%d', delimiter=',')  

# Tentukan lokasi file GeoTIFF kedua
filepath_citra2 = 'TIF koreksi/2010_CART_modif2.tif'  

# Buka file GeoTIFF kedua dengan mode baca menggunakan rasterio.open() dan atur CRS serta transformasinya
# Baca band pertama dari file raster kedua sebagai float
with rasterio.open(filepath_citra2, 'r', driver='GTiff', crs=crs, transform=transform) as src2:
    raster_t2 = src2.read(1).astype(float)  

# Tentukan nama file untuk menyimpan data raster kedua sebagai teks
inputfile_t2 = 't2.txt'  

# Simpan matriks NumPy sebagai file teks
np.savetxt(inputfile_t2, raster_t2, fmt='%d', delimiter=',')  

# Ubah data raster pertama menjadi array NumPy
array1 = np.array(raster_t1)  

# Ubah data raster kedua menjadi array NumPy
array2 = np.array(raster_t2)  

# Ratakan data raster pertama menjadi array 1 dimensi
flat_data1 = raster_t1.flatten()  

# Ratakan data raster kedua menjadi array 1 dimensi
flat_data2 = raster_t2.flatten()  

# Buat DataFrame Pandas dari data raster
df = pd.DataFrame({'LULC 2000': flat_data1, 'LULC 2010': flat_data2})  

# Buat tabel silang dari DataFrame
crosstab = pd.crosstab(df['LULC 2000'], df['LULC 2010'])  

# Definisikan klasifikasi untuk kode land use/land cover (LULC)
klasifikasi = {  
    0: 'air',
    1: 'bangunan',
    2: 'vegetasi',
    3: 'sawah',
    4: 'lahan kosong'
}

# Ubah nama kolom dan indeks berdasarkan klasifikasi
crosstab = crosstab.rename(columns=klasifikasi, index=klasifikasi)  

# Tentukan nama file untuk menyimpan tabel silang sebagai teks
output_file_tabelsilang = 'tabelsilang.txt'  

# Simpan tabel silang sebagai file teks
np.savetxt(output_file_tabelsilang, crosstab, fmt='%f', delimiter=',')  

# Cetak hasil tabel silang
print("Crosstabulation:"+"\n", crosstab, "\n")  

Crosstabulation:
 LULC 2010      air  bangunan  vegetasi  sawah  lahan kosong
LULC 2000                                                  
air           6490        52      1251      0           302
bangunan         0     13867         0      0             0
vegetasi       154       257      7430   1705          3467
sawah            0       416         0  17527             0
lahan kosong  1927      6683     11203      0         27710 



In [3]:
# Hitung jumlah baris dalam tabel silang
row_sums = crosstab.sum(axis=1)  

# Hitung matriks probabilitas
probability_matrix = crosstab.div(row_sums, axis=0)  

# Cetak matriks probabilitas
print("Matriks Probabilitas: "+"\n", probability_matrix, "\n")  

# Tentukan nama file untuk menyimpan matriks probabilitas sebagai teks
output_file_matriksprob = 'matriks_probabilitas.txt'  

# Simpan matriks probabilitas sebagai file CSV
probability_matrix.to_csv(output_file_matriksprob, float_format='%f', sep=',') 

Matriks Probabilitas: 
 LULC 2010          air  bangunan  vegetasi     sawah  lahan kosong
LULC 2000                                                         
air           0.801729  0.006424  0.154540  0.000000      0.037307
bangunan      0.000000  1.000000  0.000000  0.000000      0.000000
vegetasi      0.011834  0.019749  0.570967  0.131023      0.266426
sawah         0.000000  0.023185  0.000000  0.976815      0.000000
lahan kosong  0.040549  0.140627  0.235738  0.000000      0.583086 

